# Libraries

In [ ]:
import numpy as np
import chardet
import joblib
from collections import defaultdict
import os
import re
import json

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt 
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.colors as pc
%matplotlib inline  
import psutil
from pathlib import Path
#from Functions import *
import glob
from tqdm import tqdm
import time
import psutil
from sklearn.metrics import mean_squared_error, r2_score



# Performance

## Preparing Data


In [ ]:
# Loading Data
# ARX
ARX_train_heave_true=pd.read_csv('Results/ARX/Training and Validation/y_true_train_df_heave_nb2_nf7.csv')
ARX_train_heave_pred=pd.read_csv('Results/ARX/Training and Validation/y_pred_train_df_heave_nb2_nf7.csv')

ARX_val_heave_true=pd.read_csv('Results/ARX/Training and Validation/y_true_val_df_heave_nb2_nf7.csv')
ARX_val_heave_pred=pd.read_csv('Results/ARX/Training and Validation/y_pred_val_df_heave_nb2_nf7.csv')

ARX_train_pitch_true=pd.read_csv('Results/ARX/Training and Validation/y_true_train_df_pitch_nb2_nf7.csv')
ARX_train_pitch_pred=pd.read_csv('Results/ARX/Training and Validation/y_pred_train_df_pitch_nb2_nf7.csv')

ARX_val_pitch_true=pd.read_csv('Results/ARX/Training and Validation/y_true_val_df_pitch_nb2_nf7.csv')
ARX_val_pitch_pred=pd.read_csv('Results/ARX/Training and Validation/y_pred_val_df_pitch_nb2_nf7.csv')

ARX_train_pendulum_true=pd.read_csv('Results/ARX/Training and Validation/y_true_train_ARX_penddulum_nb4_nf2.csv')
ARX_train_pendulum_pred=pd.read_csv('Results/ARX/Training and Validation/y_pred_train_df_ARX_penddulum_nb4_nf2.csv')

ARX_val_pendulum_true=pd.read_csv('Results/ARX/Training and Validation/y_true_val_ARX_penddulum_nb4_nf2.csv')
ARX_val_pendulum_pred=pd.read_csv('Results/ARX/Training and Validation/y_pred_val_df_ARX_penddulum_nb4_nf2.csv')

# XGBoost
XGB_train_true=pd.read_csv('Results/XGB/Training and Validation/y_true_train_df_XGB_nb0_nf10.csv')
XGB_train_pred=pd.read_csv('Results/XGB/Training and Validation/y_pred_train_df_XGB_nb0_nf10.csv')

XGB_val_true=pd.read_csv('Results/XGB/Training and Validation/y_true_val_df_XGB_nb0_nf10.csv')
XGB_val_pred=pd.read_csv('Results/XGB/Training and Validation/y_pred_val_df_XGB_nb0_nf10.csv')

#LSTM
LSTM_train_true=pd.read_csv('Results/LSTM/Training and Validation/LSTM_train_true.csv')
LSTM_train_pred=pd.read_csv('Results/LSTM/Training and Validation/LSTM_train_pred.csv')

LSTM_val_true=pd.read_csv('Results/LSTM/Training and Validation/LSTM_val_true.csv')
LSTM_val_pred=pd.read_csv('Results/LSTM/Training and Validation/LSTM_val_pred.csv')





In [ ]:
# Adjust Training Data Size and index so all are same

# ARX heave
ARX_train_heave_true_adjusted = ARX_train_heave_true[:-5].reset_index(drop=True)
ARX_train_heave_pred_adjusted = ARX_train_heave_pred[:-5].reset_index(drop=True)

# ARX pitch
ARX_train_pitch_true_adjusted = ARX_train_pitch_true[:-5].reset_index(drop=True)
ARX_train_pitch_pred_adjusted = ARX_train_pitch_pred[:-5].reset_index(drop=True)

# ARX pendulum
ARX_train_pendulum_true_adjusted = ARX_train_pendulum_true[:-10].reset_index(drop=True)
ARX_train_pendulum_pred_adjusted = ARX_train_pendulum_pred[:-10].reset_index(drop=True)

# XGB
XGB_train_true_adjusted = XGB_train_true[:-2].reset_index(drop=True)
XGB_train_pred_adjusted = XGB_train_pred[:-2].reset_index(drop=True)


In [ ]:
# ARX heave
ARX_val_heave_true_adjusted = ARX_val_heave_true[5:-59].reset_index(drop=True)
ARX_val_heave_pred_adjusted = ARX_val_heave_pred[5:-59].reset_index(drop=True)

# ARX pitch
ARX_val_pitch_true_adjusted = ARX_val_pitch_true[5:-59].reset_index(drop=True)
ARX_val_pitch_pred_adjusted = ARX_val_pitch_pred[5:-59].reset_index(drop=True)

# ARX pendulum
ARX_val_pendulum_true_adjusted = ARX_val_pendulum_true[5:-64].reset_index(drop=True)
ARX_val_pendulum_pred_adjusted = ARX_val_pendulum_pred[5:-64].reset_index(drop=True)

# XGB
XGB_val_true_adjusted = XGB_val_true[5:-56].reset_index(drop=True)
XGB_val_pred_adjusted = XGB_val_pred[5:-56].reset_index(drop=True)


In [ ]:
print("Lengths of adjusted training DataFrames:")
print("ARX Heave True:", len(ARX_train_heave_true_adjusted))
print("ARX Heave Pred:", len(ARX_train_heave_pred_adjusted))
print("ARX Pitch True:", len(ARX_train_pitch_true_adjusted))
print("ARX Pitch Pred:", len(ARX_train_pitch_pred_adjusted))
print("ARX Pendulum True:", len(ARX_train_pendulum_true_adjusted))
print("ARX Pendulum Pred:", len(ARX_train_pendulum_pred_adjusted))
print("XGB True:", len(XGB_train_true_adjusted))
print("XGB Pred:", len(XGB_train_pred_adjusted))


In [ ]:
# Validate that all True data sets are identical

a = ARX_val_heave_true_adjusted['heave'].to_numpy()
b = LSTM_val_true['heave'].to_numpy()
c = XGB_val_true_adjusted['heave'].to_numpy()  # this is just 'a' again

are_all_equal = np.all((a == b) & (b == c))  # or (a == b) & (a == c)

print("All equal:", are_all_equal)


In [ ]:
Train_truth=XGB_train_true_adjusted
Val_truth=XGB_val_true_adjusted

## Training Data

In [ ]:
# Define ARX adjusted predictions
arx_preds = {
    'heave': ARX_train_heave_pred_adjusted['heave'],
    'pitch': ARX_train_pitch_pred_adjusted['pitch'],
    'pendulum': ARX_train_pendulum_pred_adjusted['pendulum'],
}


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

# Define DoFs
dofs = ['heave', 'pitch', 'pendulum']

# Initialize metrics dictionary
metrics = {}

for dof in dofs:
    y_true = Train_truth[dof]

    metrics[dof] = {
        # ARX
        'ARX_R2': r2_score(y_true, arx_preds[dof]),
        'ARX_RMSE': np.sqrt(mean_squared_error(y_true, arx_preds[dof])),

        # XGBoost
        'XGB_R2': r2_score(y_true, XGB_train_pred_adjusted[dof]),
        'XGB_RMSE': np.sqrt(mean_squared_error(y_true, XGB_train_pred_adjusted[dof])),

        # LSTM
        'LSTM_R2': r2_score(y_true, LSTM_train_pred[f'{dof}_pred']),
        'LSTM_RMSE': np.sqrt(mean_squared_error(y_true, LSTM_train_pred[f'{dof}_pred'])),
    }

# Convert to DataFrame for display
metrics_df = pd.DataFrame(metrics).T
print(metrics_df)


In [ ]:
# performnce aftr removing 1st 450 time steps 
# Define DoFs
dofs = ['heave', 'pitch', 'pendulum']

# Initialize metrics dictionary
metrics = {}

for dof in dofs:
    y_true = Train_truth[dof][450:]

    metrics[dof] = {
        # ARX
        'ARX_R2': r2_score(y_true, arx_preds[dof][450:]),
        #'ARX_RMSE': np.sqrt(mean_squared_error(y_true, arx_preds[dof][450:])),

        # XGBoost
        'XGB_R2': r2_score(y_true, XGB_train_pred_adjusted[dof][450:]),
        #'XGB_RMSE': np.sqrt(mean_squared_error(y_true, XGB_train_pred_adjusted[dof][450:])),

        # LSTM
        'LSTM_R2': r2_score(y_true, LSTM_train_pred[f'{dof}_pred'][450:]),
        #'LSTM_RMSE': np.sqrt(mean_squared_error(y_true, LSTM_train_pred[f'{dof}_pred'][450:])),
    }

# Convert to DataFrame for display
metrics_df = pd.DataFrame(metrics).T
print(metrics_df)


In [ ]:
# Define DoFs and corresponding y-axis units
dofs = ['heave', 'pitch', 'pendulum']
y_units = {
    'heave': 'm',
    'pitch': '°',
    'pendulum': '°'
}



# Loop through each DoF
for dof in dofs:
    fig = go.Figure()

    # Ground Truth (solid black)
    fig.add_trace(go.Scatter(
        y=Train_truth[dof],
        mode='lines',
        name=f'{dof.capitalize()} - Ground Truth',
        line=dict(color='black', dash='solid')
    ))

    # ARX Prediction (dashed blue)
    fig.add_trace(go.Scatter(
        y=arx_preds[dof],
        mode='lines',
        name='ARX Prediction',
        line=dict(color='blue', dash='dash')
    ))

    # XGBoost Prediction (dashed red)
    fig.add_trace(go.Scatter(
        y=XGB_train_pred_adjusted[dof],
        mode='lines',
        name='XGBoost-NARX Prediction',
        line=dict(color='red', dash='dash')
    ))

    # LSTM Prediction (dashed green)
    fig.add_trace(go.Scatter(
        y=LSTM_train_pred[f'{dof}_pred'],
        mode='lines',
        name='LSTM Prediction',
        line=dict(color='green', dash='dash')
    ))

    # Layout
    fig.update_layout(
        title=f'Training Data Predictions - {dof.capitalize()}',
        xaxis_title='Time Step',
        yaxis_title=f'{dof.capitalize()} [{y_units[dof]}]',
        template='plotly_white',
        xaxis_range=[0, 6719]
        
    )

    fig.show()


In [ ]:

# Define DoFs and corresponding y-axis units
dofs = ['heave', 'pitch', 'pendulum']
y_units = {
    'heave': 'm',
    'pitch': '°',
    'pendulum': '°'
}


# Loop through each DoF
for dof in dofs:
    truth = Train_truth[dof]

    fig = go.Figure()

    # ARX Absolute Error (blue)
    fig.add_trace(go.Scatter(
        y=np.abs(truth - arx_preds[dof]),
        mode='lines',
        name='ARX Absolute Error',
        line=dict(color='blue', dash='solid')
    ))

    # XGBoost Absolute Error (red)
    fig.add_trace(go.Scatter(
        y=np.abs(truth - XGB_train_pred_adjusted[dof]),
        mode='lines',
        name='XGBoost Absolute Error',
        line=dict(color='red', dash='solid')
    ))

    # LSTM Absolute Error (green)
    fig.add_trace(go.Scatter(
        y=np.abs(truth - LSTM_train_pred[f'{dof}_pred']),
        mode='lines',
        name='LSTM Absolute Error',
        line=dict(color='green', dash='solid')
    ))

    # Layout
    fig.update_layout(
        title=f'Training Data Absolute Prediction Error - {dof.capitalize()}',
        xaxis_title='Time Step',
        yaxis_title=f'Absolute Error [{y_units[dof]}]',
        template='plotly_white'
    )

    fig.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Define DoFs and their units
dofs = ['heave', 'pitch', 'pendulum']
y_units = {'heave': 'm', 'pitch': '°', 'pendulum': '°'}

y_ranges = {
    'heave': [0, 1],
    'pitch': [0, 0.3],
    'pendulum': [0, 0.4]  # or whatever range you want
}

# Create box plots for each DoF
for dof in dofs:
    truth = Train_truth[dof]

    # Calculate absolute errors
    arx_error = np.abs(truth - arx_preds[dof])
    xgb_error = np.abs(truth - XGB_train_pred_adjusted[dof])
    lstm_error = np.abs(truth - LSTM_train_pred[f'{dof}_pred'])

    # Create box plot
    fig = go.Figure()

    fig.add_trace(go.Box(
        y=arx_error,
        name='ARX',
        marker_color='blue',
        boxmean='sd'
    ))

    fig.add_trace(go.Box(
        y=xgb_error,
        name='XGBoost-NARX',
        marker_color='red',
        boxmean='sd'
    ))

    fig.add_trace(go.Box(
        y=lstm_error,
        name='LSTM',
        marker_color='green',
        boxmean='sd'
    ))

    # Update layout
    fig.update_layout(
        title=f'Training Data Absolute Prediction Error - {dof.capitalize()}',
        yaxis_title=f'Absolute Error [{y_units[dof]}]',
        template='plotly_white',
        width=600,     # Width in pixels
        height=400,
        yaxis=dict(range=y_ranges[dof]),
        
    )
    fig.update_layout(showlegend=False)

    # Save to file
    fig.write_image(f"Results/training plots/{dof}_box.png")

    fig.show()


## Validation Data

In [ ]:
# Define ARX adjusted predictions
arx_preds_val = {
    'heave': ARX_val_heave_pred_adjusted['heave'],
    'pitch': ARX_val_pitch_pred_adjusted['pitch'],
    'pendulum': ARX_val_pendulum_pred_adjusted['pendulum'],
}


In [ ]:
# Define DoFs
dofs = ['heave', 'pitch', 'pendulum']

# Initialize metrics dictionary
metrics = {}

for dof in dofs:
    y_true_val = Val_truth[dof]

    metrics[dof] = {
        # ARX
        'ARX_R2': r2_score(y_true_val, arx_preds_val[dof]),
        'ARX_RMSE': np.sqrt(mean_squared_error(y_true_val, arx_preds_val[dof])),

        # XGBoost
        'XGB_R2': r2_score(y_true_val, XGB_val_pred_adjusted[dof]),
        'XGB_RMSE': np.sqrt(mean_squared_error(y_true_val, XGB_val_pred_adjusted[dof])),

        # LSTM
        'LSTM_R2': r2_score(y_true_val, LSTM_val_pred[f'{dof}_pred']),
        'LSTM_RMSE': np.sqrt(mean_squared_error(y_true_val, LSTM_val_pred[f'{dof}_pred'])),
    }

# Convert to DataFrame for display
metrics_df = pd.DataFrame(metrics).T
print(metrics_df)


In [ ]:
# REMOVING 1ST 450 POINTS
# Define DoFs
dofs = ['heave', 'pitch', 'pendulum']

# Initialize metrics dictionary
metrics = {}

for dof in dofs:
    y_true_val = Val_truth[dof][450:]

    metrics[dof] = {
        # ARX
        'ARX_R2': r2_score(y_true_val, arx_preds_val[dof][450:]),
        #'ARX_RMSE': np.sqrt(mean_squared_error(y_true_val, arx_preds_val[dof][450:])),

        # XGBoost
        'XGB_R2': r2_score(y_true_val, XGB_val_pred_adjusted[dof][450:]),
        #'XGB_RMSE': np.sqrt(mean_squared_error(y_true_val, XGB_val_pred_adjusted[dof][450:])),

        # LSTM
        'LSTM_R2': r2_score(y_true_val, LSTM_val_pred[f'{dof}_pred'][450:]),
        #'LSTM_RMSE': np.sqrt(mean_squared_error(y_true_val, LSTM_val_pred[f'{dof}_pred'][450:])),
    }

# Convert to DataFrame for display
metrics_df = pd.DataFrame(metrics).T
print(metrics_df)


In [ ]:
# Define DoFs and corresponding y-axis units
dofs = ['heave', 'pitch', 'pendulum']
y_units = {
    'heave': 'm',
    'pitch': '°',
    'pendulum': '°'
}



# Loop through each DoF
for dof in dofs:
    fig = go.Figure()

    # Ground Truth (solid black)
    fig.add_trace(go.Scatter(
        y=Val_truth[dof],
        mode='lines',
        name=f'{dof.capitalize()} - Ground Truth',
        line=dict(color='black', dash='solid')
    ))

    # ARX Prediction (dashed blue)
    fig.add_trace(go.Scatter(
        y=arx_preds_val[dof],
        mode='lines',
        name='ARX Prediction',
        line=dict(color='blue', dash='dash')
    ))

    # XGBoost Prediction (dashed red)
    fig.add_trace(go.Scatter(
        y=XGB_val_pred_adjusted[dof],
        mode='lines',
        name='XGBoost-NARX Prediction',
        line=dict(color='red', dash='dash')
    ))

    # LSTM Prediction (dashed green)
    fig.add_trace(go.Scatter(
        y=LSTM_val_pred[f'{dof}_pred'],
        mode='lines',
        name='LSTM Prediction',
        line=dict(color='green', dash='dash')
    ))

    # Layout
    fig.update_layout(
        title=f'Validation Data Predictions - {dof.capitalize()}',
        xaxis_title='Time Step',
        yaxis_title=f'{dof.capitalize()} [{y_units[dof]}]',
        template='plotly_white',
        xaxis_range=[0, 6719]
    )

    fig.show()


In [ ]:

# Define DoFs and corresponding y-axis units
dofs = ['heave', 'pitch', 'pendulum']
y_units = {
    'heave': 'm',
    'pitch': '°',
    'pendulum': '°'
}


# Loop through each DoF
for dof in dofs:
    truth = Val_truth[dof]

    fig = go.Figure()

    # ARX Absolute Error (blue)
    fig.add_trace(go.Scatter(
        y=np.abs(truth - arx_preds_val[dof]),
        mode='lines',
        name='ARX Absolute Error',
        line=dict(color='blue', dash='solid')
    ))

    # XGBoost Absolute Error (red)
    fig.add_trace(go.Scatter(
        y=np.abs(truth - XGB_val_pred_adjusted[dof]),
        mode='lines',
        name='XGBoost Absolute Error',
        line=dict(color='red', dash='solid')
    ))

    # LSTM Absolute Error (green)
    fig.add_trace(go.Scatter(
        y=np.abs(truth - LSTM_val_pred[f'{dof}_pred']),
        mode='lines',
        name='LSTM Absolute Error',
        line=dict(color='green', dash='solid')
    ))

    # Layout
    fig.update_layout(
        title=f'Validation Data Absolute Prediction Error - {dof.capitalize()}',
        xaxis_title='Time Step',
        yaxis_title=f'Absolute Error [{y_units[dof]}]',
        template='plotly_white'
    )

    fig.show()


In [ ]:
# Define DoFs and their units
dofs = ['heave', 'pitch', 'pendulum']
y_units = {'heave': 'm', 'pitch': '°', 'pendulum': '°'}



# Create box plots for each DoF
for dof in dofs:
    truth = Val_truth[dof]

    # Calculate absolute errors
    arx_error = np.abs(truth - arx_preds_val[dof])
    xgb_error = np.abs(truth - XGB_val_pred_adjusted[dof])
    lstm_error = np.abs(truth - LSTM_val_pred[f'{dof}_pred'])

    # Create box plot
    fig = go.Figure()

    fig.add_trace(go.Box(
        y=arx_error,
        name='ARX',
        marker_color='blue',
        boxmean='sd'
    ))

    fig.add_trace(go.Box(
        y=xgb_error,
        name='XGBoost-NARX',
        marker_color='red',
        boxmean='sd'
    ))

    fig.add_trace(go.Box(
        y=lstm_error,
        name='LSTM',
        marker_color='green',
        boxmean='sd'
    ))

    # Update layout
    fig.update_layout(
        title=f'Validation Data Absolute Prediction Error - {dof.capitalize()}',
        yaxis_title=f'Absolute Error [{y_units[dof]}]',
        template='plotly_white',
        width=600,     # Width in pixels
        height=400,
                yaxis=dict(range=y_ranges[dof]),

        
    )
    fig.update_layout(showlegend=False)

    fig.write_image(f"Results/Validation plots/{dof}_box.png")

    fig.show()


# Loading testing data

In [ ]:
# Set the folder path
folder_path = 'Results/ARX/Testing/heave'

# Create an empty dictionary to hold the DataFrames
csv_data_ARX_heave = {}

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        # Use filename (without .csv) as key
        key = os.path.splitext(filename)[0]
        csv_data_ARX_heave[key] = pd.read_csv(file_path)

# Example: Access one of the loaded DataFrames
print(csv_data_ARX_heave.keys())  # Shows all loaded file names


In [ ]:
# Set the folder path
folder_path = 'Results/ARX/Testing/pitch'

# Create an empty dictionary to hold the DataFrames
csv_data_ARX_pitch = {}

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        # Use filename (without .csv) as key
        key = os.path.splitext(filename)[0]
        csv_data_ARX_pitch[key] = pd.read_csv(file_path)

# Example: Access one of the loaded DataFrames
print(csv_data_ARX_pitch.keys())  # Shows all loaded file names


In [ ]:
# Set the folder path
folder_path = 'Results/ARX/Testing/pendulum'

# Create an empty dictionary to hold the DataFrames
csv_data_ARX_pendulum = {}

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        # Use filename (without .csv) as key
        key = os.path.splitext(filename)[0]
        csv_data_ARX_pendulum[key] = pd.read_csv(file_path)

# Example: Access one of the loaded DataFrames
print(csv_data_ARX_pendulum.keys())  # Shows all loaded file names


In [ ]:
# Set the folder path
folder_path = 'Results/XGB/Testing'

# Create an empty dictionary to hold the DataFrames
csv_data_XGB = {}

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        # Use filename (without .csv) as key
        key = os.path.splitext(filename)[0]
        csv_data_XGB[key] = pd.read_csv(file_path)

# Example: Access one of the loaded DataFrames
print(csv_data_XGB.keys())  # Shows all loaded file names


In [ ]:
# Set the folder path
folder_path = 'Results/LSTM/Testing'

# Create an empty dictionary to hold the DataFrames
csv_data_LSTM = {}

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        # Use filename (without .csv) as key
        key = os.path.splitext(filename)[0]
        csv_data_LSTM[key] = pd.read_csv(file_path)

# Example: Access one of the loaded DataFrames
print(csv_data_LSTM.keys())  # Shows all loaded file names


In [ ]:
# T10p2s_Hs1m
ARX_heave_T10p2s_Hs1m_true = csv_data_ARX_heave['y_true_test_heave_T10p2s_Hs1m'][5:].reset_index(drop=True)
ARX_heave_T10p2s_Hs1m_pred = csv_data_ARX_heave['y_pred_test_heave_T10p2s_Hs1m'][5:].reset_index(drop=True)

# T10p2s_Hs4m
ARX_heave_T10p2s_Hs4m_true = csv_data_ARX_heave['y_true_test_heave_T10p2s_Hs4m'][5:].reset_index(drop=True)
ARX_heave_T10p2s_Hs4m_pred = csv_data_ARX_heave['y_pred_test_heave_T10p2s_Hs4m'][5:].reset_index(drop=True)

# T4p5s_Hs1m
ARX_heave_T4p5s_Hs1m_true = csv_data_ARX_heave['y_true_test_heave_T4p5s_Hs1m'][5:].reset_index(drop=True)
ARX_heave_T4p5s_Hs1m_pred = csv_data_ARX_heave['y_pred_test_heave_T4p5s_Hs1m'][5:].reset_index(drop=True)

# T4p5s_Hs2m
ARX_heave_T4p5s_Hs2m_true = csv_data_ARX_heave['y_true_test_heave_T4p5s_Hs2m'][5:].reset_index(drop=True)
ARX_heave_T4p5s_Hs2m_pred = csv_data_ARX_heave['y_pred_test_heave_T4p5s_Hs2m'][5:].reset_index(drop=True)

# Tp10p2_Hs2m
ARX_heave_Tp10p2_Hs2m_true = csv_data_ARX_heave['y_true_test_heave_Tp10p2_Hs2m'][5:].reset_index(drop=True)
ARX_heave_Tp10p2_Hs2m_pred = csv_data_ARX_heave['y_pred_test_heave_Tp10p2_Hs2m'][5:].reset_index(drop=True)

# Tp6p8s_Hs1m
ARX_heave_Tp6p8s_Hs1m_true = csv_data_ARX_heave['y_true_test_heave_Tp6p8s_Hs1m'][5:].reset_index(drop=True)
ARX_heave_Tp6p8s_Hs1m_pred = csv_data_ARX_heave['y_pred_test_heave_Tp6p8s_Hs1m'][5:].reset_index(drop=True)

# Tp6p8s_Hs2m
ARX_heave_Tp6p8s_Hs2m_true = csv_data_ARX_heave['y_true_test_heave_Tp6p8s_Hs2m'][5:].reset_index(drop=True)
ARX_heave_Tp6p8s_Hs2m_pred = csv_data_ARX_heave['y_pred_test_heave_Tp6p8s_Hs2m'][5:].reset_index(drop=True)

# Tp6p8s_Hs4m
ARX_heave_Tp6p8s_Hs4m_true = csv_data_ARX_heave['y_true_test_heave_Tp6p8s_Hs4m'][5:].reset_index(drop=True)
ARX_heave_Tp6p8s_Hs4m_pred = csv_data_ARX_heave['y_pred_test_heave_Tp6p8s_Hs4m'][5:].reset_index(drop=True)

# Tp6p8s_Hs6m
ARX_heave_Tp6p8s_Hs6m_true = csv_data_ARX_heave['y_true_test_heave_Tp6p8s_Hs6m'][5:].reset_index(drop=True)
ARX_heave_Tp6p8s_Hs6m_pred = csv_data_ARX_heave['y_pred_test_heave_Tp6p8s_Hs6m'][5:].reset_index(drop=True)

# Tp6p8s_Hs8m
ARX_heave_Tp6p8s_Hs8m_true = csv_data_ARX_heave['y_true_test_heave_Tp6p8s_Hs8m'][5:].reset_index(drop=True)
ARX_heave_Tp6p8s_Hs8m_pred = csv_data_ARX_heave['y_pred_test_heave_Tp6p8s_Hs8m'][5:].reset_index(drop=True)


In [ ]:
# T10p2s_Hs1m
ARX_pitch_T10p2s_Hs1m_true = csv_data_ARX_pitch['y_true_test_pitch_T10p2s_Hs1m'][5:].reset_index(drop=True)
ARX_pitch_T10p2s_Hs1m_pred = csv_data_ARX_pitch['y_pred_test_pitch_T10p2s_Hs1m'][5:].reset_index(drop=True)

# T10p2s_Hs4m
ARX_pitch_T10p2s_Hs4m_true = csv_data_ARX_pitch['y_true_test_pitch_T10p2s_Hs4m'][5:].reset_index(drop=True)
ARX_pitch_T10p2s_Hs4m_pred = csv_data_ARX_pitch['y_pred_test_pitch_T10p2s_Hs4m'][5:].reset_index(drop=True)

# T4p5s_Hs1m
ARX_pitch_T4p5s_Hs1m_true = csv_data_ARX_pitch['y_true_test_pitch_T4p5s_Hs1m'][5:].reset_index(drop=True)
ARX_pitch_T4p5s_Hs1m_pred = csv_data_ARX_pitch['y_pred_test_pitch_T4p5s_Hs1m'][5:].reset_index(drop=True)

# T4p5s_Hs2m
ARX_pitch_T4p5s_Hs2m_true = csv_data_ARX_pitch['y_true_test_pitch_T4p5s_Hs2m'][5:].reset_index(drop=True)
ARX_pitch_T4p5s_Hs2m_pred = csv_data_ARX_pitch['y_pred_test_pitch_T4p5s_Hs2m'][5:].reset_index(drop=True)

# Tp10p2_Hs2m
ARX_pitch_Tp10p2_Hs2m_true = csv_data_ARX_pitch['y_true_test_pitch_Tp10p2_Hs2m'][5:].reset_index(drop=True)
ARX_pitch_Tp10p2_Hs2m_pred = csv_data_ARX_pitch['y_pred_test_pitch_Tp10p2_Hs2m'][5:].reset_index(drop=True)

# Tp6p8s_Hs1m
ARX_pitch_Tp6p8s_Hs1m_true = csv_data_ARX_pitch['y_true_test_pitch_Tp6p8s_Hs1m'][5:].reset_index(drop=True)
ARX_pitch_Tp6p8s_Hs1m_pred = csv_data_ARX_pitch['y_pred_test_pitch_Tp6p8s_Hs1m'][5:].reset_index(drop=True)

# Tp6p8s_Hs2m
ARX_pitch_Tp6p8s_Hs2m_true = csv_data_ARX_pitch['y_true_test_pitch_Tp6p8s_Hs2m'][5:].reset_index(drop=True)
ARX_pitch_Tp6p8s_Hs2m_pred = csv_data_ARX_pitch['y_pred_test_pitch_Tp6p8s_Hs2m'][5:].reset_index(drop=True)

# Tp6p8s_Hs4m
ARX_pitch_Tp6p8s_Hs4m_true = csv_data_ARX_pitch['y_true_test_pitch_Tp6p8s_Hs4m'][5:].reset_index(drop=True)
ARX_pitch_Tp6p8s_Hs4m_pred = csv_data_ARX_pitch['y_pred_test_pitch_Tp6p8s_Hs4m'][5:].reset_index(drop=True)

# Tp6p8s_Hs6m
ARX_pitch_Tp6p8s_Hs6m_true = csv_data_ARX_pitch['y_true_test_pitch_Tp6p8s_Hs6m'][5:].reset_index(drop=True)
ARX_pitch_Tp6p8s_Hs6m_pred = csv_data_ARX_pitch['y_pred_test_pitch_Tp6p8s_Hs6m'][5:].reset_index(drop=True)

# Tp6p8s_Hs8m
ARX_pitch_Tp6p8s_Hs8m_true = csv_data_ARX_pitch['y_true_test_pitch_Tp6p8s_Hs8m'][5:].reset_index(drop=True)
ARX_pitch_Tp6p8s_Hs8m_pred = csv_data_ARX_pitch['y_pred_test_pitch_Tp6p8s_Hs8m'][5:].reset_index(drop=True)


In [ ]:
# T10p2s_Hs1m
ARX_pendulum_T10p2s_Hs1m_true = csv_data_ARX_pendulum['y_true_test_pitch_T10p2s_Hs1m'][5:].reset_index(drop=True)
ARX_pendulum_T10p2s_Hs1m_pred = csv_data_ARX_pendulum['y_pred_test_dfT10p2s_Hs1m'][5:].reset_index(drop=True)

# T10p2s_Hs4m
ARX_pendulum_T10p2s_Hs4m_true = csv_data_ARX_pendulum['y_true_test_pitch_T10p2s_Hs4m'][5:].reset_index(drop=True)
ARX_pendulum_T10p2s_Hs4m_pred = csv_data_ARX_pendulum['y_pred_test_dfT10p2s_Hs4m'][5:].reset_index(drop=True)

# T4p5s_Hs1m
ARX_pendulum_T4p5s_Hs1m_true = csv_data_ARX_pendulum['y_true_test_pitch_T4p5s_Hs1m'][5:].reset_index(drop=True)
ARX_pendulum_T4p5s_Hs1m_pred = csv_data_ARX_pendulum['y_pred_test_dfT4p5s_Hs1m'][5:].reset_index(drop=True)

# T4p5s_Hs2m
ARX_pendulum_T4p5s_Hs2m_true = csv_data_ARX_pendulum['y_true_test_pitch_T4p5s_Hs2m'][5:].reset_index(drop=True)
ARX_pendulum_T4p5s_Hs2m_pred = csv_data_ARX_pendulum['y_pred_test_dfT4p5s_Hs2m'][5:].reset_index(drop=True)

# Tp10p2_Hs2m
ARX_pendulum_Tp10p2_Hs2m_true = csv_data_ARX_pendulum['y_true_test_pitch_Tp10p2_Hs2m'][5:].reset_index(drop=True)
ARX_pendulum_Tp10p2_Hs2m_pred = csv_data_ARX_pendulum['y_pred_test_dfTp10p2_Hs2m'][5:].reset_index(drop=True)

# Tp6p8s_Hs1m
ARX_pendulum_Tp6p8s_Hs1m_true = csv_data_ARX_pendulum['y_true_test_pitch_Tp6p8s_Hs1m'][5:].reset_index(drop=True)
ARX_pendulum_Tp6p8s_Hs1m_pred = csv_data_ARX_pendulum['y_pred_test_dfTp6p8s_Hs1m'][5:].reset_index(drop=True)

# Tp6p8s_Hs2m
ARX_pendulum_Tp6p8s_Hs2m_true = csv_data_ARX_pendulum['y_true_test_pitch_Tp6p8s_Hs2m'][5:].reset_index(drop=True)
ARX_pendulum_Tp6p8s_Hs2m_pred = csv_data_ARX_pendulum['y_pred_test_dfTp6p8s_Hs2m'][5:].reset_index(drop=True)

# Tp6p8s_Hs4m
ARX_pendulum_Tp6p8s_Hs4m_true = csv_data_ARX_pendulum['y_true_test_pitch_Tp6p8s_Hs4m'][5:].reset_index(drop=True)
ARX_pendulum_Tp6p8s_Hs4m_pred = csv_data_ARX_pendulum['y_pred_test_dfTp6p8s_Hs4m'][5:].reset_index(drop=True)

# Tp6p8s_Hs6m
ARX_pendulum_Tp6p8s_Hs6m_true = csv_data_ARX_pendulum['y_true_test_pitch_Tp6p8s_Hs6m'][5:].reset_index(drop=True)
ARX_pendulum_Tp6p8s_Hs6m_pred = csv_data_ARX_pendulum['y_pred_test_dfTp6p8s_Hs6m'][5:].reset_index(drop=True)

# Tp6p8s_Hs8m
ARX_pendulum_Tp6p8s_Hs8m_true = csv_data_ARX_pendulum['y_true_test_pitch_Tp6p8s_Hs8m'][5:].reset_index(drop=True)
ARX_pendulum_Tp6p8s_Hs8m_pred = csv_data_ARX_pendulum['y_pred_test_dfTp6p8s_Hs8m'][5:].reset_index(drop=True)


In [ ]:
# T10p2s_Hs1m
XGB_T10p2s_Hs1m_true = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_T10p2s_Hs1m_y_true'][5:].reset_index(drop=True)
XGB_T10p2s_Hs1m_pred = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_T10p2s_Hs1m_y_pred'][5:].reset_index(drop=True)

# T10p2s_Hs4m
XGB_T10p2s_Hs4m_true = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_T10p2s_Hs4m_y_true'][5:].reset_index(drop=True)
XGB_T10p2s_Hs4m_pred = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_T10p2s_Hs4m_y_pred'][5:].reset_index(drop=True)

# T4p5s_Hs1m
XGB_T4p5s_Hs1m_true = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_T4p5s_Hs1m_y_true'][5:].reset_index(drop=True)
XGB_T4p5s_Hs1m_pred = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_T4p5s_Hs1m_y_pred'][5:].reset_index(drop=True)

# T4p5s_Hs2m
XGB_T4p5s_Hs2m_true = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_T4p5s_Hs2m_y_true'][5:].reset_index(drop=True)
XGB_T4p5s_Hs2m_pred = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_T4p5s_Hs2m_y_pred'][5:].reset_index(drop=True)

# Tp10p2_Hs2m
XGB_Tp10p2_Hs2m_true = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp10p2_Hs2m_y_true'][5:].reset_index(drop=True)
XGB_Tp10p2_Hs2m_pred = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp10p2_Hs2m_y_pred'][5:].reset_index(drop=True)

# Tp6p8s_Hs1m
XGB_Tp6p8s_Hs1m_true = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp6p8s_Hs1m_y_true'][5:].reset_index(drop=True)
XGB_Tp6p8s_Hs1m_pred = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp6p8s_Hs1m_y_pred'][5:].reset_index(drop=True)

# Tp6p8s_Hs2m
XGB_Tp6p8s_Hs2m_true = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp6p8s_Hs2m_y_true'][5:].reset_index(drop=True)
XGB_Tp6p8s_Hs2m_pred = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp6p8s_Hs2m_y_pred'][5:].reset_index(drop=True)

# Tp6p8s_Hs4m
XGB_Tp6p8s_Hs4m_true = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp6p8s_Hs4m_y_true'][5:].reset_index(drop=True)
XGB_Tp6p8s_Hs4m_pred = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp6p8s_Hs4m_y_pred'][5:].reset_index(drop=True)

# Tp6p8s_Hs6m
XGB_Tp6p8s_Hs6m_true = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp6p8s_Hs6m_y_true'][5:].reset_index(drop=True)
XGB_Tp6p8s_Hs6m_pred = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp6p8s_Hs6m_y_pred'][5:].reset_index(drop=True)

# Tp6p8s_Hs8m
XGB_Tp6p8s_Hs8m_true = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp6p8s_Hs8m_y_true'][5:].reset_index(drop=True)
XGB_Tp6p8s_Hs8m_pred = csv_data_XGB['Xgboost_3dof_ver2_extra_na2_nb0_nf10.joblib_case_Tp6p8s_Hs8m_y_pred'][5:].reset_index(drop=True)


In [ ]:
# T10p2s_Hs1m
LSTM_T10p2s_Hs1m_true = csv_data_LSTM['LSTM_true_T10p2s_Hs1m'][:].reset_index(drop=True)
LSTM_T10p2s_Hs1m_pred = csv_data_LSTM['LSTM_pred_T10p2s_Hs1m'][:].reset_index(drop=True)

# T10p2s_Hs4m
LSTM_T10p2s_Hs4m_true = csv_data_LSTM['LSTM_true_T10p2s_Hs4m'][:].reset_index(drop=True)
LSTM_T10p2s_Hs4m_pred = csv_data_LSTM['LSTM_pred_T10p2s_Hs4m'][:].reset_index(drop=True)

# T4p5s_Hs1m
LSTM_T4p5s_Hs1m_true = csv_data_LSTM['LSTM_true_T4p5s_Hs1m'][:].reset_index(drop=True)
LSTM_T4p5s_Hs1m_pred = csv_data_LSTM['LSTM_pred_T4p5s_Hs1m'][:].reset_index(drop=True)

# T4p5s_Hs2m
LSTM_T4p5s_Hs2m_true = csv_data_LSTM['LSTM_true_T4p5s_Hs2m'][:].reset_index(drop=True)
LSTM_T4p5s_Hs2m_pred = csv_data_LSTM['LSTM_pred_T4p5s_Hs2m'][:].reset_index(drop=True)

# Tp10p2_Hs2m
LSTM_Tp10p2_Hs2m_true = csv_data_LSTM['LSTM_true_Tp10p2_Hs2m'][:].reset_index(drop=True)
LSTM_Tp10p2_Hs2m_pred = csv_data_LSTM['LSTM_pred_Tp10p2_Hs2m'][:].reset_index(drop=True)

# Tp6p8s_Hs1m
LSTM_Tp6p8s_Hs1m_true = csv_data_LSTM['LSTM_true_Tp6p8s_Hs1m'][:].reset_index(drop=True)
LSTM_Tp6p8s_Hs1m_pred = csv_data_LSTM['LSTM_pred_Tp6p8s_Hs1m'][:].reset_index(drop=True)

# Tp6p8s_Hs2m
LSTM_Tp6p8s_Hs2m_true = csv_data_LSTM['LSTM_true_Tp6p8s_Hs2m'][:].reset_index(drop=True)
LSTM_Tp6p8s_Hs2m_pred = csv_data_LSTM['LSTM_pred_Tp6p8s_Hs2m'][:].reset_index(drop=True)

# Tp6p8s_Hs4m
LSTM_Tp6p8s_Hs4m_true = csv_data_LSTM['LSTM_true_Tp6p8s_Hs4m'][:].reset_index(drop=True)
LSTM_Tp6p8s_Hs4m_pred = csv_data_LSTM['LSTM_pred_Tp6p8s_Hs4m'][:].reset_index(drop=True)

# Tp6p8s_Hs6m
LSTM_Tp6p8s_Hs6m_true = csv_data_LSTM['LSTM_true_Tp6p8s_Hs6m'][:].reset_index(drop=True)
LSTM_Tp6p8s_Hs6m_pred = csv_data_LSTM['LSTM_pred_Tp6p8s_Hs6m'][:].reset_index(drop=True)

# Tp6p8s_Hs8m
LSTM_Tp6p8s_Hs8m_true = csv_data_LSTM['LSTM_true_Tp6p8s_Hs8m'][:].reset_index(drop=True)
LSTM_Tp6p8s_Hs8m_pred = csv_data_LSTM['LSTM_pred_Tp6p8s_Hs8m'][:].reset_index(drop=True)


In [ ]:
# Remove extra samples at the end to match batch size
def adjust_for_batch(X):
    n = (X.shape[0] // 64) * 64
    return X[:n]

In [ ]:
# List of variable names as strings
data_names = [
    "ARX_heave_T10p2s_Hs1m_true", "ARX_heave_T10p2s_Hs1m_pred",
    "ARX_heave_T10p2s_Hs4m_true", "ARX_heave_T10p2s_Hs4m_pred",
    "ARX_heave_T4p5s_Hs1m_true", "ARX_heave_T4p5s_Hs1m_pred",
    "ARX_heave_T4p5s_Hs2m_true", "ARX_heave_T4p5s_Hs2m_pred",
    "ARX_heave_Tp10p2_Hs2m_true", "ARX_heave_Tp10p2_Hs2m_pred",
    "ARX_heave_Tp6p8s_Hs1m_true", "ARX_heave_Tp6p8s_Hs1m_pred",
    "ARX_heave_Tp6p8s_Hs2m_true", "ARX_heave_Tp6p8s_Hs2m_pred",
    "ARX_heave_Tp6p8s_Hs4m_true", "ARX_heave_Tp6p8s_Hs4m_pred",
    "ARX_heave_Tp6p8s_Hs6m_true", "ARX_heave_Tp6p8s_Hs6m_pred",
    "ARX_heave_Tp6p8s_Hs8m_true", "ARX_heave_Tp6p8s_Hs8m_pred",
    "ARX_pitch_T10p2s_Hs1m_true", "ARX_pitch_T10p2s_Hs1m_pred",
    "ARX_pitch_T10p2s_Hs4m_true", "ARX_pitch_T10p2s_Hs4m_pred",
    "ARX_pitch_T4p5s_Hs1m_true", "ARX_pitch_T4p5s_Hs1m_pred",
    "ARX_pitch_T4p5s_Hs2m_true", "ARX_pitch_T4p5s_Hs2m_pred",
    "ARX_pitch_Tp10p2_Hs2m_true", "ARX_pitch_Tp10p2_Hs2m_pred",
    "ARX_pitch_Tp6p8s_Hs1m_true", "ARX_pitch_Tp6p8s_Hs1m_pred",
    "ARX_pitch_Tp6p8s_Hs2m_true", "ARX_pitch_Tp6p8s_Hs2m_pred",
    "ARX_pitch_Tp6p8s_Hs4m_true", "ARX_pitch_Tp6p8s_Hs4m_pred",
    "ARX_pitch_Tp6p8s_Hs6m_true", "ARX_pitch_Tp6p8s_Hs6m_pred",
    "ARX_pitch_Tp6p8s_Hs8m_true", "ARX_pitch_Tp6p8s_Hs8m_pred",
    "ARX_pendulum_T10p2s_Hs1m_true", "ARX_pendulum_T10p2s_Hs1m_pred",
    "ARX_pendulum_T10p2s_Hs4m_true", "ARX_pendulum_T10p2s_Hs4m_pred",
    "ARX_pendulum_T4p5s_Hs1m_true", "ARX_pendulum_T4p5s_Hs1m_pred",
    "ARX_pendulum_T4p5s_Hs2m_true", "ARX_pendulum_T4p5s_Hs2m_pred",
    "ARX_pendulum_Tp10p2_Hs2m_true", "ARX_pendulum_Tp10p2_Hs2m_pred",
    "ARX_pendulum_Tp6p8s_Hs1m_true", "ARX_pendulum_Tp6p8s_Hs1m_pred",
    "ARX_pendulum_Tp6p8s_Hs2m_true", "ARX_pendulum_Tp6p8s_Hs2m_pred",
    "ARX_pendulum_Tp6p8s_Hs4m_true", "ARX_pendulum_Tp6p8s_Hs4m_pred",
    "ARX_pendulum_Tp6p8s_Hs6m_true", "ARX_pendulum_Tp6p8s_Hs6m_pred",
    "ARX_pendulum_Tp6p8s_Hs8m_true", "ARX_pendulum_Tp6p8s_Hs8m_pred",
    "XGB_T10p2s_Hs1m_true", "XGB_T10p2s_Hs1m_pred",
    "XGB_T10p2s_Hs4m_true", "XGB_T10p2s_Hs4m_pred",
    "XGB_T4p5s_Hs1m_true", "XGB_T4p5s_Hs1m_pred",
    "XGB_T4p5s_Hs2m_true", "XGB_T4p5s_Hs2m_pred",
    "XGB_Tp10p2_Hs2m_true", "XGB_Tp10p2_Hs2m_pred",
    "XGB_Tp6p8s_Hs1m_true", "XGB_Tp6p8s_Hs1m_pred",
    "XGB_Tp6p8s_Hs2m_true", "XGB_Tp6p8s_Hs2m_pred",
    "XGB_Tp6p8s_Hs4m_true", "XGB_Tp6p8s_Hs4m_pred",
    "XGB_Tp6p8s_Hs6m_true", "XGB_Tp6p8s_Hs6m_pred",
    "XGB_Tp6p8s_Hs8m_true", "XGB_Tp6p8s_Hs8m_pred"
]

# Apply the function and store as new variables with "_adjusted" suffix
for name in data_names:
    original = globals()[name]
    adjusted = adjust_for_batch(original)
    globals()[name + "_adjusted"] = adjusted


In [ ]:
#checking
# List of test cases to check
test_cases = [
    "T10p2s_Hs1m",
    "T10p2s_Hs4m",
    "T4p5s_Hs1m",
    "T4p5s_Hs2m",
    "Tp10p2_Hs2m",
    "Tp6p8s_Hs1m",
    "Tp6p8s_Hs2m",
    "Tp6p8s_Hs4m",
    "Tp6p8s_Hs6m",
    "Tp6p8s_Hs8m"
]

for test in test_cases:
    try:
        a = globals()[f"ARX_heave_{test}_true_adjusted"]['heave'].to_numpy()
        b = globals()[f"LSTM_{test}_true"]['heave'].to_numpy()
        c = globals()[f"XGB_{test}_true_adjusted"]['heave'].to_numpy()

        are_all_equal = np.all((a == b) & (b == c))
        print(f"{test}: All equal? {are_all_equal}")

    except KeyError as e:
        print(f"{test}: Missing variable {e}")
    except Exception as ex:
        print(f"{test}: Error - {ex}")


In [ ]:
#checking
# List of test cases to check
test_cases = [
    "T10p2s_Hs1m",
    "T10p2s_Hs4m",
    "T4p5s_Hs1m",
    "T4p5s_Hs2m",
    "Tp10p2_Hs2m",
    "Tp6p8s_Hs1m",
    "Tp6p8s_Hs2m",
    "Tp6p8s_Hs4m",
    "Tp6p8s_Hs6m",
    "Tp6p8s_Hs8m"
]

for test in test_cases:
    try:
        a = globals()[f"ARX_pitch_{test}_true_adjusted"]['pitch'].to_numpy()
        b = globals()[f"LSTM_{test}_true"]['pitch'].to_numpy()
        c = globals()[f"XGB_{test}_true_adjusted"]['pitch'].to_numpy()

        are_all_equal = np.all((a == b) & (b == c))
        print(f"{test}: All equal? {are_all_equal}")

    except KeyError as e:
        print(f"{test}: Missing variable {e}")
    except Exception as ex:
        print(f"{test}: Error - {ex}")


In [ ]:
#checking
# List of test cases to check
test_cases = [
    "T10p2s_Hs1m",
    "T10p2s_Hs4m",
    "T4p5s_Hs1m",
    "T4p5s_Hs2m",
    "Tp10p2_Hs2m",
    "Tp6p8s_Hs1m",
    "Tp6p8s_Hs2m",
    "Tp6p8s_Hs4m",
    "Tp6p8s_Hs6m",
    "Tp6p8s_Hs8m"
]

for test in test_cases:
    try:
        a = globals()[f"ARX_pendulum_{test}_true_adjusted"]['pendulum'].to_numpy()
        b = globals()[f"LSTM_{test}_true"]['pendulum'].to_numpy()
        c = globals()[f"XGB_{test}_true_adjusted"]['pendulum'].to_numpy()

        are_all_equal = np.all((a == b) & (b == c))
        print(f"{test}: All equal? {are_all_equal}")

    except KeyError as e:
        print(f"{test}: Missing variable {e}")
    except Exception as ex:
        print(f"{test}: Error - {ex}")


## Testing Case

In [ ]:
# Define ARX adjusted predictions
arx_preds_test = {
    'heave': ARX_heave_Tp6p8s_Hs2m_pred_adjusted['heave'],
    'pitch': ARX_pitch_Tp6p8s_Hs2m_pred_adjusted['pitch'],
    'pendulum': ARX_pendulum_Tp6p8s_Hs2m_pred_adjusted['pendulum'],
}


In [ ]:
# Define DoFs
dofs = ['heave', 'pitch', 'pendulum']

# Initialize metrics dictionary
metrics = {}

for dof in dofs:
    y_true_val = LSTM_Tp6p8s_Hs2m_true[dof]

    metrics[dof] = {
        # ARX
        'ARX_R2': r2_score(y_true_val, arx_preds_test[dof]),
        'ARX_RMSE': np.sqrt(mean_squared_error(y_true_val, arx_preds_test[dof])),

        # XGBoost
        'XGB_R2': r2_score(y_true_val, XGB_Tp6p8s_Hs2m_pred_adjusted[dof]),
        'XGB_RMSE': np.sqrt(mean_squared_error(y_true_val, XGB_Tp6p8s_Hs2m_pred_adjusted[dof])),

        # LSTM
        'LSTM_R2': r2_score(y_true_val, LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred']),
        'LSTM_RMSE': np.sqrt(mean_squared_error(y_true_val, LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred'])),
    }

# Convert to DataFrame for display
metrics_df = pd.DataFrame(metrics).T
print(metrics_df)


In [ ]:
# removing 1st 450 time steps
# Define DoFs
dofs = ['heave', 'pitch', 'pendulum']

# Initialize metrics dictionary
metrics = {}

for dof in dofs:
    y_true_val = LSTM_Tp6p8s_Hs2m_true[dof][450:]

    metrics[dof] = {
        # ARX
        'ARX_R2': r2_score(y_true_val, arx_preds_test[dof][450:]),
       # 'ARX_RMSE': np.sqrt(mean_squared_error(y_true_val, arx_preds_test[dof][450:])),

        # XGBoost
        'XGB_R2': r2_score(y_true_val, XGB_Tp6p8s_Hs2m_pred_adjusted[dof][450:]),
        #'XGB_RMSE': np.sqrt(mean_squared_error(y_true_val, XGB_Tp6p8s_Hs2m_pred_adjusted[dof][450:])),

        # LSTM
        'LSTM_R2': r2_score(y_true_val, LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred'][450:]),
        #'LSTM_RMSE': np.sqrt(mean_squared_error(y_true_val, LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred'][450:])),
    }

# Convert to DataFrame for display
metrics_df = pd.DataFrame(metrics).T
print(metrics_df)


In [ ]:
# Define DoFs and corresponding y-axis units
dofs = ['heave', 'pitch', 'pendulum']
y_units = {
    'heave': 'm',
    'pitch': '°',
    'pendulum': '°'
}



# Loop through each DoF
for dof in dofs:
    fig = go.Figure()

    # Ground Truth (solid black)
    fig.add_trace(go.Scatter(
        y=LSTM_Tp6p8s_Hs2m_true[dof],
        mode='lines',
        name=f'{dof.capitalize()} - Ground Truth',
        line=dict(color='black', dash='solid')
    ))

    # ARX Prediction (dashed blue)
    fig.add_trace(go.Scatter(
        y=arx_preds_test[dof],
        mode='lines',
        name='ARX Prediction',
        line=dict(color='blue', dash='dash')
    ))

    # XGBoost Prediction (dashed red)
    fig.add_trace(go.Scatter(
        y=XGB_Tp6p8s_Hs2m_pred_adjusted[dof],
        mode='lines',
        name='XGBoost-NARX Prediction',
        line=dict(color='red', dash='dash')
    ))

    # LSTM Prediction (dashed green)
    fig.add_trace(go.Scatter(
        y=LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred'],
        mode='lines',
        name='LSTM Prediction',
        line=dict(color='green', dash='dash')
    ))

    # Layout
    fig.update_layout(
        title=f'Testing Data Predictions - {dof.capitalize()}',
        xaxis_title='Time Step',
        yaxis_title=f'{dof.capitalize()} [{y_units[dof]}]',
        template='plotly_white'
    )

    fig.show()


In [ ]:

# Define DoFs and corresponding y-axis units
dofs = ['heave', 'pitch', 'pendulum']
y_units = {
    'heave': 'm',
    'pitch': '°',
    'pendulum': '°'
}


# Loop through each DoF
for dof in dofs:
    truth = LSTM_Tp6p8s_Hs2m_true[dof]

    fig = go.Figure()

    # ARX Absolute Error (blue)
    fig.add_trace(go.Scatter(
        y=np.abs(truth - arx_preds_test[dof]),
        mode='lines',
        name='ARX Absolute Error',
        line=dict(color='blue', dash='solid')
    ))

    # XGBoost Absolute Error (red)
    fig.add_trace(go.Scatter(
        y=np.abs(truth - XGB_Tp6p8s_Hs2m_pred_adjusted[dof]),
        mode='lines',
        name='XGBoost Absolute Error',
        line=dict(color='red', dash='solid')
    ))

    # LSTM Absolute Error (green)
    fig.add_trace(go.Scatter(
        y=np.abs(truth - LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred']),
        mode='lines',
        name='LSTM Absolute Error',
        line=dict(color='green', dash='solid')
    ))

    # Layout
    fig.update_layout(
        title=f'Testing Data Absolute Prediction Error - {dof.capitalize()}',
        xaxis_title='Time Step',
        yaxis_title=f'Absolute Error [{y_units[dof]}]',
        template='plotly_white'
    )
    
    fig.show()


In [ ]:
# Define DoFs and their units
dofs = ['heave', 'pitch', 'pendulum']
y_units = {'heave': 'm', 'pitch': '°', 'pendulum': '°'}



# Create box plots for each DoF
for dof in dofs:
    truth = LSTM_Tp6p8s_Hs2m_true[dof]

    # Calculate absolute errors
    arx_error = np.abs(truth - arx_preds_test[dof])
    xgb_error = np.abs(truth - XGB_Tp6p8s_Hs2m_pred_adjusted[dof])
    lstm_error = np.abs(truth - LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred'])

    # Create box plot
    fig = go.Figure()

    fig.add_trace(go.Box(
        y=arx_error,
        name='ARX',
        marker_color='blue',
        boxmean='sd'
    ))

    fig.add_trace(go.Box(
        y=xgb_error,
        name='XGBoost-NARX',
        marker_color='red',
        boxmean='sd'
    ))

    fig.add_trace(go.Box(
        y=lstm_error,
        name='LSTM',
        marker_color='green',
        boxmean='sd'
    ))

    # Update layout
    fig.update_layout(
        title=f'Testing Data Absolute Prediction Error - {dof.capitalize()}',
        yaxis_title=f'Absolute Error [{y_units[dof]}]',
        template='plotly_white',
        width=600,     # Width in pixels
        height=400,
        yaxis=dict(range=y_ranges[dof]),

        
    )
    fig.update_layout(showlegend=False)

    fig.write_image(f"Results/testing plots/{dof}_box.png")

    fig.show()


## Extrapolation

In [ ]:
test_case_pretty_map = {
    'T4p5s_Hs1m': 'Tp = 4.5 s, Hs = 1 m',
    'T4p5s_Hs2m': 'Tp = 4.5 s, Hs = 2 m',
    'Tp6p8s_Hs1m': 'Tp = 6.8 s, Hs = 1 m',
    'Tp6p8s_Hs2m': 'Tp = 6.8 s, Hs = 2 m',
    'Tp6p8s_Hs4m': 'Tp = 6.8 s, Hs = 4 m',
    'Tp6p8s_Hs6m': 'Tp = 6.8 s, Hs = 6 m',
    'Tp6p8s_Hs8m': 'Tp = 6.8 s, Hs = 8 m',
    'T10p2s_Hs1m': 'Tp = 10.2 s, Hs = 1 m',
    'Tp10p2_Hs2m': 'Tp = 10.2 s, Hs = 2 m',
    'T10p2s_Hs4m': 'Tp = 10.2 s, Hs = 4 m'
}


In [ ]:

# Define DoFs and their y-axis units
dofs = ['heave', 'pitch', 'pendulum']
y_units = {
    'heave': 'm',
    'pitch': '°',
    'pendulum': '°'
}

# Test cases to evaluate
test_cases = [
    "T10p2s_Hs1m",
    "T10p2s_Hs4m",
    "T4p5s_Hs1m",
    "T4p5s_Hs2m",
    "Tp10p2_Hs2m",
    "Tp6p8s_Hs1m",
    "Tp6p8s_Hs2m",

    "Tp6p8s_Hs4m",
    "Tp6p8s_Hs6m",
    "Tp6p8s_Hs8m"
]

In [ ]:
# Store all metrics
all_metrics = []

for case in test_cases:
    for dof in dofs:
        try:
            # Ground truth from LSTM
            y_true = globals()[f"LSTM_{case}_true"][dof]

            # ARX prediction
            y_arx = globals()[f"ARX_{dof}_{case}_pred_adjusted"]
            arx_r2 = r2_score(y_true, y_arx)
            arx_rmse = np.sqrt(mean_squared_error(y_true, y_arx))

            # XGB prediction
            y_xgb = globals()[f"XGB_{case}_pred_adjusted"][dof]
            xgb_r2 = r2_score(y_true, y_xgb)
            xgb_rmse = np.sqrt(mean_squared_error(y_true, y_xgb))

            # LSTM prediction
            y_lstm = globals()[f"LSTM_{case}_pred"][f"{dof}_pred"]
            lstm_r2 = r2_score(y_true, y_lstm)
            lstm_rmse = np.sqrt(mean_squared_error(y_true, y_lstm))

            # Store results
            all_metrics.append({
                "Case": case,
                "DoF": dof,
                "ARX_R2": arx_r2,
                "ARX_RMSE": arx_rmse,
                "XGB_R2": xgb_r2,
                "XGB_RMSE": xgb_rmse,
                "LSTM_R2": lstm_r2,
                "LSTM_RMSE": lstm_rmse
            })

        except KeyError as e:
            print(f"[Missing] {case} - {dof}: {e}")
        except Exception as ex:
            print(f"[Error] {case} - {dof}: {ex}")

# Convert to DataFrame
metrics_df = pd.DataFrame(all_metrics)

# Display
print(metrics_df)

In [ ]:
test_cases_ordered = [
    "T4p5s_Hs1m",
    "T4p5s_Hs2m",
    
    "Tp6p8s_Hs1m",
    "Tp6p8s_Hs2m",
    "Tp6p8s_Hs4m",
    "Tp6p8s_Hs6m",
    "Tp6p8s_Hs8m",
    "T10p2s_Hs1m",
    "Tp10p2_Hs2m",
    "T10p2s_Hs4m"
]


In [ ]:
# Your mapping of case names to pretty labels
test_case_pretty_map = {
    'T4p5s_Hs1m': 'Tp = 4.5 s, Hs = 1 m',
    'T4p5s_Hs2m': 'Tp = 4.5 s, Hs = 2 m',
    'Tp6p8s_Hs1m': 'Tp = 6.8 s, Hs = 1 m',
    'Tp6p8s_Hs2m': 'Tp = 6.8 s, Hs = 2 m',
    'Tp6p8s_Hs4m': 'Tp = 6.8 s, Hs = 4 m',
    'Tp6p8s_Hs6m': 'Tp = 6.8 s, Hs = 6 m',
    'Tp6p8s_Hs8m': 'Tp = 6.8 s, Hs = 8 m',
    'T10p2s_Hs1m': 'Tp = 10.2 s, Hs = 1 m',
    'Tp10p2_Hs2m': 'Tp = 10.2 s, Hs = 2 m',
    'T10p2s_Hs4m': 'Tp = 10.2 s, Hs = 4 m'
}

# Pretty x-axis labels
pretty_labels = [test_case_pretty_map[case] for case in test_cases_ordered]

In [ ]:
test_cases = [
    "T4p5s_Hs1m",
    "T4p5s_Hs2m",
    "Tp6p8s_Hs4m",
    "Tp6p8s_Hs1m",
    "Tp6p8s_Hs6m",
    "Tp6p8s_Hs8m",
    "T10p2s_Hs1m",
    "Tp10p2_Hs2m",
    "T10p2s_Hs4m"
]

In [ ]:
highlight_case = "Tp6p8s_Hs2m"

# Model settings: color, marker symbol, and line style
model_info = {
    "ARX": {"color": "blue", "symbol": "x", "dash": "dash"},
    "XGB": {"color": "red", "symbol": "circle", "dash": "dot"},
    "LSTM": {"color": "green", "symbol": "square", "dash": "solid"},
}

# Loop through DoFs
for dof in dofs:
    metric_dof = metrics_df[metrics_df["DoF"] == dof]
    fig = make_subplots(rows=1, cols=2, subplot_titles=("RMSE", "R² Score"))

    rmse_unit = "m" if dof == "heave" else "°"

    for metric_type, col, title, subplot_col in [
        ("RMSE", "_RMSE", "RMSE", 1),
        ("R2", "_R2", "R² Score", 2)
    ]:
        for model, props in model_info.items():
            y_vals = []
            sizes = []
            opacities = []

            for case in test_cases_ordered:
                val = metric_dof.loc[metric_dof["Case"] == case, f"{model}_{metric_type}"]
                y_vals.append(val.values[0] if not val.empty else None)

                if case == highlight_case:
                    sizes.append(8)
                else:
                    sizes.append(8)


            # Main line + markers trace
            fig.add_trace(go.Scatter(
                x=pretty_labels,
                y=y_vals,
                mode="lines+markers",
                name=f"{'XGBoost-NARX' if model == 'XGB' else model} ",
                marker=dict(
                    symbol=props["symbol"],
                    color=props["color"],
                    size=sizes,
                    opacity=opacities,
                    line=dict(width=2, color="rgba(255,255,255,0.4)")
                ),
                line=dict(color=props["color"], dash=props["dash"]),
                showlegend=(subplot_col == 1)
            ), row=1, col=subplot_col)

            # Add blue glow for the highlight case only
            if highlight_case in test_cases_ordered:
                idx = test_cases_ordered.index(highlight_case)
                fig.add_trace(go.Scatter(
                    x=[pretty_labels[idx]],
                    y=[y_vals[idx]],
                    mode='markers',
                    showlegend=False,
                    marker=dict(
                        color="rgba(255, 255, 0, 0.4)", 
                        size=10,
                        symbol=props["symbol"],
                        line=dict(width=0)
                    )
                ), row=1, col=subplot_col)

    fig.update_layout(
        title_text=f"Model Performance Across Sea States – {dof.capitalize()}",
        template="plotly_white",
        height=500
    )

    fig.update_yaxes(title_text=f"RMSE [{rmse_unit}]", row=1, col=1)
    fig.update_yaxes(title_text="R² Score", row=1, col=2)
    fig.update_xaxes(title_text="Sea State", tickangle=-45)

    fig.show()


In [ ]:


for case in test_cases:
    for dof in dofs:
        fig = go.Figure()
        success = True

        # Define variable base names
        arx_var = f'ARX_{dof}_{case}_true_adjusted'
        xgb_var = f'XGB_{case}_true_adjusted'
        lstm_var = f'LSTM_{case}_true'
        
        arx_pred_var = f'ARX_{dof}_{case}_pred_adjusted'
        xgb_pred_var = f'XGB_{case}_pred_adjusted'
        lstm_pred_var = f'LSTM_{case}_pred'

        try:
            # Ground truth from ARX or LSTM or XGB (any source is OK if identical)
            truth = globals()[arx_var][dof]

            # ARX prediction
            arx_pred = globals()[arx_pred_var][dof]
            fig.add_trace(go.Scatter(
                y=arx_pred,
                mode='lines',
                name='ARX Prediction',
                line=dict(color='blue', dash='dash')
            ))

            # XGB prediction
            xgb_pred = globals()[xgb_pred_var][dof]
            fig.add_trace(go.Scatter(
                y=xgb_pred,
                mode='lines',
                name='XGBoost-NARX Prediction',
                line=dict(color='red', dash='dash')
            ))

            # LSTM prediction
            lstm_pred = globals()[lstm_pred_var][f'{dof}_pred']
            fig.add_trace(go.Scatter(
                y=lstm_pred,
                mode='lines',
                name='LSTM Prediction',
                line=dict(color='green', dash='dash')
            ))

            # Ground Truth
            fig.add_trace(go.Scatter(
                y=truth,
                mode='lines',
                name=f'{dof.capitalize()} - Ground Truth',
                line=dict(color='black', dash='solid')
            ))

        except KeyError as e:
            print(f"[Missing] {case} {dof}: {e}")
            success = False

        if success:
            fig.update_layout(
                title=f'Sea State: {test_case_pretty_map.get(case, case)} – DoF: {dof.capitalize()}',
                xaxis_title='Time Step',
                yaxis_title=f'{dof.capitalize()} [{y_units[dof]}]',
                template='plotly_white'
            )
            fig.show()
            fig.write_image(f"Annex/Testing/original/{case}_{dof}.png", scale=2)



# uncertnity analysis

In [ ]:
# Define DoFs
dofs = ['heave', 'pitch', 'pendulum']

# Initialize metrics dictionary
nonconformity_scores = {}

for dof in dofs:
    y_true_val = Val_truth[dof]

    nonconformity_scores[dof] = {
        # ARX
        'ARX_abs_error':abs(y_true_val - arx_preds_val[dof]),
        

        # XGBoost
        'XGB_abs_error': abs(y_true_val - XGB_val_pred_adjusted[dof]),
        
        # LSTM,
        'LSTM_abs_error': abs(y_true_val - LSTM_val_pred[f'{dof}_pred'])
        
    }

# Convert to DataFrame for display
metricnonconformity_scoress_df = pd.DataFrame(nonconformity_scores)
metricnonconformity_scoress_df


In [ ]:
# Set confidence level
confidence_level = 0.95  

# Calculate thresholds
thresholds = {}
for dof in dofs:
    thresholds[dof] = {
        'ARX': np.quantile(nonconformity_scores[dof]['ARX_abs_error'], confidence_level),
        'XGB': np.quantile(nonconformity_scores[dof]['XGB_abs_error'], confidence_level),
        'LSTM': np.quantile(nonconformity_scores[dof]['LSTM_abs_error'], confidence_level)
    }

# Display thresholds
thresholds_df_95 = pd.DataFrame(thresholds)
print("95% Confidence Level Thresholds:")
print(thresholds_df_95)

In [ ]:
# Set confidence level
confidence_level = 0.99  

# Calculate thresholds
thresholds_99 = {}
for dof in dofs:
    thresholds_99[dof] = {
        'ARX': np.quantile(nonconformity_scores[dof]['ARX_abs_error'], confidence_level),
        'XGB': np.quantile(nonconformity_scores[dof]['XGB_abs_error'], confidence_level),
        'LSTM': np.quantile(nonconformity_scores[dof]['LSTM_abs_error'], confidence_level)
    }

# Display thresholds
thresholds_df_99 = pd.DataFrame(thresholds_99)
print("99% Confidence Level Thresholds:")
print(thresholds_df_99)

In [ ]:
prediction_intervals_95 = {}

for dof in dofs:
    prediction_intervals_95[dof] = {}
    
    # Get test predictions 
    arx_test_pred = arx_preds_test[dof]  #  ARX test predictions
    xgb_test_pred = XGB_Tp6p8s_Hs2m_pred_adjusted[dof]  #  XGBoost test predictions  
    lstm_test_pred = LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred']  #  LSTM test predictions
    
    # Calculate prediction intervals
    prediction_intervals_95[dof]['ARX'] = {
        'lower': arx_test_pred - thresholds_df_95[dof]['ARX'],
        'upper': arx_test_pred + thresholds_df_95[dof]['ARX'],
        'prediction': arx_test_pred
    }
    
    prediction_intervals_95[dof]['XGB'] = {
        'lower': xgb_test_pred - thresholds_df_95[dof]['XGB'],
        'upper': xgb_test_pred + thresholds_df_95[dof]['XGB'],
        'prediction': xgb_test_pred
    }
    
    prediction_intervals_95[dof]['LSTM'] = {
        'lower': lstm_test_pred - thresholds_df_95[dof]['LSTM'],
        'upper': lstm_test_pred + thresholds_df_95[dof]['LSTM'],
        'prediction': lstm_test_pred
    }

In [ ]:

prediction_intervals_99 = {}

for dof in dofs:
    prediction_intervals_99[dof] = {}
    
    # Get test predictions 
    arx_test_pred = arx_preds_test[dof]  #  ARX test predictions
    xgb_test_pred = XGB_Tp6p8s_Hs2m_pred_adjusted[dof]  #  XGBoost test predictions  
    lstm_test_pred = LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred']  #  LSTM test predictions
    
    # Calculate prediction intervals
    prediction_intervals_99[dof]['ARX'] = {
        'lower': arx_test_pred - thresholds_df_99[dof]['ARX'],
        'upper': arx_test_pred + thresholds_df_99[dof]['ARX'],
        'prediction': arx_test_pred
    }
    
    prediction_intervals_99[dof]['XGB'] = {
        'lower': xgb_test_pred - thresholds_df_99[dof]['XGB'],
        'upper': xgb_test_pred + thresholds_df_99[dof]['XGB'],
        'prediction': xgb_test_pred
    }
    
    prediction_intervals_99[dof]['LSTM'] = {
        'lower': lstm_test_pred - thresholds_df_99[dof]['LSTM'],
        'upper': lstm_test_pred + thresholds_df_99[dof]['LSTM'],
        'prediction': lstm_test_pred
    }

In [ ]:
prediction_intervals = {}

for dof in dofs:
    prediction_intervals[dof] = {}
    
    # Get test predictions 
    arx_test_pred = arx_preds_test[dof]  #  ARX test predictions
    xgb_test_pred = XGB_Tp6p8s_Hs2m_pred_adjusted[dof]  #  XGBoost test predictions  
    lstm_test_pred = LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred']  #  LSTM test predictions
    
    # Calculate prediction intervals
    prediction_intervals[dof]['ARX'] = {
        'lower': arx_test_pred - thresholds_df_99[dof]['ARX'],
        'upper': arx_test_pred + thresholds_df_99[dof]['ARX'],
        'prediction': arx_test_pred
    }
    
    prediction_intervals[dof]['XGB'] = {
        'lower': xgb_test_pred - thresholds_df_99[dof]['XGB'],
        'upper': xgb_test_pred + thresholds_df_99[dof]['XGB'],
        'prediction': xgb_test_pred
    }
    
    prediction_intervals[dof]['LSTM'] = {
        'lower': lstm_test_pred - thresholds_df_99[dof]['LSTM'],
        'upper': lstm_test_pred + thresholds_df_99[dof]['LSTM'],
        'prediction': lstm_test_pred
    }

In [ ]:

# Define DoFs and corresponding y-axis units
dofs = ['heave', 'pitch', 'pendulum']
y_units = {
    'heave': 'm',
    'pitch': '°',
    'pendulum': '°'
}

# Set confidence level for prediction intervals
confidence_level = 0.99  

# Loop through each DoF
for dof in dofs:
    # Create subplots for ARX, XGBoost, and LSTM in a single row
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=["ARX", "XGBoost-NARX", "LSTM"],
        shared_yaxes=True
    )

    # Add ARX Prediction (dashed blue)
    fig.add_trace(go.Scatter(
        x=np.arange(1000),  # First 1000 time steps
        y=arx_preds_test[dof][:1000],  # ARX Prediction
        mode='lines',
        name='ARX ',
        line=dict(color='blue', dash='dash')
    ), row=1, col=1)

    # Add ARX Confidence Interval
    fig.add_trace(go.Scatter(
        x=np.arange(1000),  # First 1000 time steps
        y=arx_preds_test[dof][:1000] + thresholds_99[dof]['ARX'],  # Upper bound for ARX
        fill=None,
        mode='lines',
        line=dict(color='blue', dash='dot'),
        showlegend=False
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=np.arange(1000),
        y=arx_preds_test[dof][:1000] - thresholds_99[dof]['ARX'],  # Lower bound for ARX
        fill='tonexty',
        mode='lines',
        line=dict(color='blue', dash='dot'),
        name='ARX Confidence Interval (99%)',
        fillcolor='rgba(0, 0, 255, 0.2)',
        showlegend=False
    ), row=1, col=1)

    # Add XGBoost Prediction (dashed red)
    fig.add_trace(go.Scatter(
        x=np.arange(1000),  # First 1000 time steps
        y=XGB_Tp6p8s_Hs2m_pred_adjusted[dof][:1000],  # XGBoost Prediction
        mode='lines',
        name='XGBoost-NARX ',
        line=dict(color='red', dash='dash')
    ), row=1, col=2)

    # Add XGBoost Confidence Interval
    fig.add_trace(go.Scatter(
        x=np.arange(1000),  # First 1000 time steps
        y=XGB_Tp6p8s_Hs2m_pred_adjusted[dof][:1000] + thresholds_99[dof]['XGB'],  # Upper bound for XGBoost
        fill=None,
        mode='lines',
        line=dict(color='red', dash='dot'),
        showlegend=False
    ), row=1, col=2)
    
    fig.add_trace(go.Scatter(
        x=np.arange(1000),
        y=XGB_Tp6p8s_Hs2m_pred_adjusted[dof][:1000] - thresholds_99[dof]['XGB'],  # Lower bound for XGBoost
        fill='tonexty',
        mode='lines',
        line=dict(color='red', dash='dot'),
        name='XGBoost-NARX Confidence Interval (99%)',
        fillcolor='rgba(255, 0, 0, 0.2)',
        showlegend=False
    ), row=1, col=2)

    # Add LSTM Prediction (dashed green)
    fig.add_trace(go.Scatter(
        x=np.arange(1000),  # First 1000 time steps
        y=LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred'][:1000],  # LSTM Prediction
        mode='lines',
        name='LSTM ',
        line=dict(color='green', dash='dash')
    ), row=1, col=3)

    # Add LSTM Confidence Interval
    fig.add_trace(go.Scatter(
        x=np.arange(1000),  # First 1000 time steps
        y=LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred'][:1000] + thresholds_99[dof]['LSTM'],  # Upper bound for LSTM
        fill=None,
        mode='lines',
        line=dict(color='green', dash='dot'),
        showlegend=False
    ), row=1, col=3)

    fig.add_trace(go.Scatter(
        x=np.arange(1000),
        y=LSTM_Tp6p8s_Hs2m_pred[f'{dof}_pred'][:1000] - thresholds_99[dof]['LSTM'],  # Lower bound for LSTM
        fill='tonexty',
        mode='lines',
        line=dict(color='green', dash='dot'),
        name='LSTM Confidence Interval',
        fillcolor='rgba(0, 255, 0, 0.2)',
        showlegend=False
    ), row=1, col=3)

    # Ground Truth (solid black) - Same for all models
    fig.add_trace(go.Scatter(
        x=np.arange(1000),  # First 1000 time steps
        y=LSTM_Tp6p8s_Hs2m_true[dof][:1000],  # Ground Truth
        mode='lines',
        name='Ground Truth',
        line=dict(color='black', dash='solid')
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=np.arange(1000),
        y=LSTM_Tp6p8s_Hs2m_true[dof][:1000],  # Ground Truth
        mode='lines',
        showlegend=False,
        name=f'{dof.capitalize()} - Ground Truth',
        line=dict(color='black', dash='solid')
    ), row=1, col=2)

    fig.add_trace(go.Scatter(
        x=np.arange(1000),
        y=LSTM_Tp6p8s_Hs2m_true[dof][:1000],  # Ground Truth
        mode='lines',
        showlegend=False,
        name=f'{dof.capitalize()}Ground Truth',
        line=dict(color='black', dash='solid')
    ), row=1, col=3)

    # Layout
    fig.update_layout(
        title=f'99% Confidence Intervals for Models Predictions across Testing Data - {dof.capitalize()}',
        xaxis_title='Time Step',
        yaxis_title=f'{dof.capitalize()} [{y_units[dof]}]',
        template='plotly_white',
        showlegend=True,
        height=400,
        width=800
    )
    
    output_dir = "Results/uncertanity/"
    os.makedirs(output_dir, exist_ok=True)
    fig.write_image(f"{output_dir}{dof}_uncertanity.png")
    # Show the figure

